In [13]:
# Demo functionality of networkx-backed textgraph.
import re, sys
import graphviz
import matplotlib.pyplot as plt
from pathlib import Path

import networkx as nx
from networkx import nx_agraph
# see also: https://nxv.readthedocs.io/en/latest/#

from textgrid import textgrid, textgraph, util

In [2]:
# Read textgrid.
grid_file = Path.home() / \
    'Projects/SCOTUSProductionPlanning/textgrids_tagged/2013/12-574.TextGrid'
dat = textgrid.read(grid_file)

# Convert to graph.
graph = textgraph.to_graph(dat)
print(graph)

# Add local speaking rate to each phone/word node.
textgraph.speaking_rate(graph, side='before', window=1000.0)
textgraph.speaking_rate(graph, side='after', window=1000.0)

# Convert back to dataframe.
dat_ = textgraph.to_dat(graph)
print(dat_)


Textgrid /home/cwilso23/Projects/SCOTUSProductionPlanning/textgrids_tagged/2013/12-574.TextGrid
* Speakers: ['Jeffrey S. Bucholtz', 'Ruth Bader Ginsburg', 'Stephen G. Breyer', 'Samuel A. Alito, Jr.', 'Melissa Arbus Sherry', '<null>', 'Sonia Sotomayor', 'Anthony M. Kennedy', 'Elena Kagan', 'Antonin Scalia', 'John G. Roberts, Jr.', 'Thomas C. Goldstein']
* Tiers: ['phones', 'words', 'pos']

dat shape: (65_574, 7)
┌───────────────────────────┬──────────────────────┬────────┬───────┬──────────┬──────────┬────────┐
│ filename                  ┆ speaker              ┆ tier   ┆ label ┆ start    ┆ end      ┆ dur_ms │
│ ---                       ┆ ---                  ┆ ---    ┆ ---   ┆ ---      ┆ ---      ┆ ---    │
│ str                       ┆ str                  ┆ str    ┆ str   ┆ f64      ┆ f64      ┆ f64    │
╞═══════════════════════════╪══════════════════════╪════════╪═══════╪══════════╪══════════╪════════╡
│ /home/cwilso23/Projects/S ┆ <null>               ┆ pos    ┆ PRON  ┆ 1.25     

In [31]:
# Visualize subgraph.
word = textgraph.get_nodes(graph, tier='word')[20]
prec = textgraph.get_prec(graph, word)
succ = textgraph.get_succ(graph, word)
phones = textgraph.get_phones(graph, [prec, word, succ])
nodes = [word, prec, succ] + util.flatten(phones)
node_ids = [idx for (idx, _) in nodes]
subgraph = graph.subgraph(node_ids).copy()
edges = list(subgraph.edges(data = True))
for (u, v, d) in edges:
    d_label = d.get('label')
    if d_label in ['word', 'prec']:
        subgraph.remove_edge(u, v)

fig = nx_agraph.view_pygraphviz(subgraph, prog="dot", args="[rankdir=LR]")
#fig

OSError: Error: dot: can't open [rankdir=LR]: No such file or directory
